In [1]:
import json
import numpy as np
import pandas as pd
from dateutil import parser
import preprocess_hop_log, merge_hop_tick, encode_brokerid
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [2]:
# path_hoplog = '../raw_data/hopLogger/12/'
# path_ticklog = '../raw_data/tickLogger/12/'
# path_mergeddf = '../preprocessed_data/12/'

DIR_NAME = 210
BROKER = '50003'

path_hoplog = '../../wowsan/log/hopLogger/' + str(DIR_NAME) + '/'
path_ticklog = '../../wowsan/log/tickLogger/' + str(DIR_NAME) + '/'
path_mergeddf = '../preprocessed_data_informer/' + str(DIR_NAME) + '/'


## Hop Log 전처리

In [3]:
# json 파일을 열고 각 줄을 개별적으로 파싱
# path = '../raw_data/hopLogger/116/'
data = []
with open(path_hoplog + BROKER + '.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터를 DataFrame으로 변환
hop_df_raw = pd.DataFrame(data)
type(hop_df_raw)

pandas.core.frame.DataFrame

In [4]:
hop_df_raw

,Node,PerformanceInfo,level,msg,time
0,localhost:50003,"[{'BrokerId': 'localhost:50001', 'Cpu': '64.282521', 'Memory': '22675456', 'QueueLength': '0', 'QueueTime': '0.303324', 'AverageQueueTime': '', 'ServiceTime': '0.000000', 'AverageServiceTime': '', 'ResponseTime': '0.303324', 'InterArrivalTime': '0.000000', 'Throughput': '3.296801', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:35:52.7619299 +0900 KST m=+5.818396501'}, {'BrokerId': 'localhost:50002', 'Cpu': '53.477604', 'Memory': '23056384', 'QueueLength': '0', 'QueueTime': '0.311918', 'AverageQueueTime': '', 'ServiceTime': '0.000000', 'AverageServiceTime': '', 'ResponseTime': '0.311918', 'InterArrivalTime': '0.000000', 'Throughput': '3.205973', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:35:53.3877958 +0900 KST m=+6.508136701'}, {'BrokerId': 'localhost:50003', 'Cpu': '71.543500', 'Memory': '22896640', 'QueueLength': '0', 'QueueTime': '0.308376', 'AverageQueueTime': '0.308376', 'ServiceTime': '0.000000', 'AverageServiceTime': '0.000000', 'ResponseTime': '0.308376', 'InterArrivalTime': '0.000000', 'Throughput': '3.242796', 'AverageThroughput': '3.242796', 'Timestamp': '2024-11-30 17:35:54.0443138 +0900 KST m=+7.150063301'}]",info,Advertisement(apple > 0.25195215418436445) 194f50a8-90e8-45b7-8c3c-bed7fb62e514,2024-11-30 17:35:54
1,localhost:50003,"[{'BrokerId': 'localhost:50004', 'Cpu': '66.722926', 'Memory': '23662592', 'QueueLength': '0', 'QueueTime': '0.100927', 'AverageQueueTime': '', 'ServiceTime': '0.493589', 'AverageServiceTime': '', 'ResponseTime': '0.594516', 'InterArrivalTime': '0.559047', 'Throughput': '1.682040', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:35:55.5773986 +0900 KST m=+8.676233501'}, {'BrokerId': 'localhost:50003', 'Cpu': '71.700718', 'Memory': '23171072', 'QueueLength': '0', 'QueueTime': '0.303256', 'AverageQueueTime': '0.305816', 'ServiceTime': '0.622440', 'AverageServiceTime': '0.622440', 'ResponseTime': '0.925696', 'InterArrivalTime': '1.280409', 'Throughput': '1.080268', 'AverageThroughput': '1.077289', 'Timestamp': '2024-11-30 17:35:56.6000112 +0900 KST m=+9.705760701'}]",info,Advertisement(apple > 0.8905283893938075) cd793b01-1cf5-4817-98b8-2f6804b7d137,2024-11-30 17:35:56
2,localhost:50003,"[{'BrokerId': 'localhost:50005', 'Cpu': '67.067395', 'Memory': '22700032', 'QueueLength': '0', 'QueueTime': '0.126521', 'AverageQueueTime': '', 'ServiceTime': '0.141435', 'AverageServiceTime': '', 'ResponseTime': '0.267956', 'InterArrivalTime': '0.944930', 'Throughput': '3.731953', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:35:57.813104 +0900 KST m=+10.936167001'}, {'BrokerId': 'localhost:50004', 'Cpu': '69.896775', 'Memory': '24137728', 'QueueLength': '0', 'QueueTime': '0.113377', 'AverageQueueTime': '', 'ServiceTime': '0.115075', 'AverageServiceTime': '', 'ResponseTime': '0.228452', 'InterArrivalTime': '0.971101', 'Throughput': '4.377291', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:35:58.3561582 +0900 KST m=+11.454993101'}, {'BrokerId': 'localhost:50003', 'Cpu': '70.441196', 'Memory': '23486464', 'QueueLength': '0', 'QueueTime': '0.329153', 'AverageQueueTime': '0.313595', 'ServiceTime': '0.723655', 'AverageServiceTime': '0.673048', 'ResponseTime': '1.052808', 'InterArrivalTime': '1.671994', 'Throughput': '0.949840', 'AverageThroughput': '1.013538', 'Timestamp': '2024-11-30 17:35:59.0810738 +0900 KST m=+12.186823301'}]",info,Advertisement(apple > 0.9865251742875035) 4c923870-0d6b-4411-b9be-0ccdd3e6031d,2024-11-30 17:35:59
3,localhost:50003,"[{'BrokerId': 'localhost:50004', 'Cpu': '67.996619', 'Memory': '24391680', 'QueueLength': '0', 'QueueTime': '0.141062', 'AverageQueueTime': '', 'ServiceTime': '0.518791', 'AverageServiceTime': '', 'ResponseTime': '0.659852', 'InterArrivalTime': '1.207823', 'Throughput': '1.515490', 'AverageThroughput': '', 'Timestamp': '2024-11-30 17:36:00.5385549 +0900 KST m=+13.637389801'}, {'BrokerId': 'localhost:50003', 'Cpu': '64.739492', 'Memory': '23744512', 'QueueLength': '0', 'Queu

In [5]:
hop_df = preprocess_hop_log.parse_performance_info(hop_df_raw)

In [6]:
hop_df

,time,Node,level,msg,HopCount,AverageThroughput_0,ResponseTime_0,Timestamp_0,BrokerId_0,ServiceTime_0,Cpu_0,QueueLength_0,Memory_0,AverageServiceTime_0,InterArrivalTime_0,QueueTime_0,AverageQueueTime_0,Throughput_0,AverageThroughput_1,ResponseTime_1,Timestamp_1,BrokerId_1,ServiceTime_1,Cpu_1,QueueLength_1,Memory_1,AverageServiceTime_1,InterArrivalTime_1,QueueTime_1,AverageQueueTime_1,Throughput_1,AverageThroughput_2,ResponseTime_2,Timestamp_2,BrokerId_2,ServiceTime_2,Cpu_2,QueueLength_2,Memory_2,AverageServiceTime_2,InterArrivalTime_2,QueueTime_2,AverageQueueTime_2,Throughput_2
0,2024-11-30 17:35:54,localhost:50003,info,Advertisement(apple > 0.25195215418436445) 194f50a8-90e8-45b7-8c3c-bed7fb62e514,3,3.242796,0.308376,2024-11-30 17:35:54.0443138 +0900 KST m=+7.150063301,localhost:50003,0.000000,71.543500,0,22896640,0.000000,0.000000,0.308376,0.308376,3.242796,,0.311918,2024-11-30 17:35:53.3877958 +0900 KST m=+6.508136701,localhost:50002,0.000000,53.477604,0,23056384,,0.000000,0.311918,,3.205973,,0.303324,2024-11-30 17:35:52.7619299 +0900 KST m=+5.818396501,localhost:50001,0.000000,64.282521,0,22675456,,0.000000,0.303324,,3.296801
1,2024-11-30 17:35:56,localhost:50003,info,Advertisement(apple > 0.8905283893938075) cd793b01-1cf5-4817-98b8-2f6804b7d137,2,1.077289,0.925696,2024-11-30 17:35:56.6000112 +0900 KST m=+9.705760701,localhost:50003,0.622440,71.700718,0,23171072,0.622440,1.280409,0.303256,0.305816,1.080268,,0.594516,2024-11-30 17:35:55.5773986 +0900 KST m=+8.676233501,localhost:50004,0.493589,66.722926,0,23662592,,0.559047,0.100927,,1.682040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-11-30 17:35:59,localhost:50003,info,Advertisement(apple > 0.9865251742875035) 4c923870-0d6b-4411-b9be-0ccdd3e6031d,3,1.013538,1.052808,2024-11-30 17:35:59.0810738 +0900 KST m=+12.186823301,localhost:50003,0.723655,70.441196,0,23486464,0.673048,1.671994,0.329153,0.313595,0.949840,,0.228452,2024-11-30 17:35:58.3561582 +0900 KST m=+11.454993101,localhost:50004,0.115075,69.896775,0,24137728,,0.971101,0.113377,,4.377291,,0.267956,2024-11-30 17:35:57.813104 +0900 KST m=+10.936167001,localhost:50005,0.141435,67.067395,0,22700032,,0.944930,0.126521,,3.731953
3,2024-11-30 17:36:01,localhost:50003,info,Advertisement(apple > 0.6275339485574908) c5c491be-e864-4cc9-a729-803843b160ac,2,0.962491,1.146745,2024-11-30 17:36:01.4125202 +0900 KST m=+14.518269701,localhost:50003,0.820683,64.739492,0,23744512,0.722260,1.837630,0.326061,0.316711,0.872034,,0.659852,2024-11-30 17:36:00.5385549 +0900 KST m=+13.637389801,localhost:50004,0.518791,67.996619,0,24391680,,1.207823,0.141062,,1.515490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-11-30 17:36:06,localhost:50003,info,Advertisement(apple > 0.9646406184803333) 09eae1c4-2297-4467-914f-5d78ad86c9ee,2,0.937117,1.154278,2024-11-30 17:36:06.3848418 +0900 KST m=+19.490591301,localhost:50003,0.823665,58.656242,0,23986176,0.747611,2.463658,0.330613,0.319492,0.866342,,0.260359,2024-11-30 17:36:05.664143 +0900 KST m=+18.762977901,localhost:50004,0.137538,61.591855,0,24760320,,1.597563,0.122821,,3.840855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6966,2024-11-30 19:35:42,localhost:50003,info,Advertisement(apple > 0.3009694439421689) 68e848c3-aa65-46a7-8469-031f0cb1904b,3,0.021763,0.715067,2024-11-30 19:35:42.810599 +0900 KST m=+7195.916348501,localhost:50003,0.514997,52.598151,0,42082304,0.814160,1.031847,0.200070,45.135267,1.398471,,3528.266437,2024-11-30 19:35:42.2959388 +0900 KST m=+7195.394773701,localhost:50004,0.100027,64.071701,33080,77500416,,0.105465,3528.166410,,0.000283,,0.220075,2024-11-30 18:36:53.8174627 +0900 KST m=+3666.940525701,localhost:50005,0.109571,64.559301,1,39182336,,1.050810,0.110505,,4.543899
6967,2024-11-30 19:35:43,localhost:50003,info,Advertisement(apple > 0.9005249814991677) ca13b

### BrokerId_n 컬럼 인코딩

In [7]:
hop_df = encode_brokerid.encode_brokerid(hop_df)

In [8]:
hop_df

,time,Node,level,msg,HopCount,AverageThroughput_0,ResponseTime_0,Timestamp_0,BrokerId_0,ServiceTime_0,Cpu_0,QueueLength_0,Memory_0,AverageServiceTime_0,InterArrivalTime_0,QueueTime_0,AverageQueueTime_0,Throughput_0,AverageThroughput_1,ResponseTime_1,Timestamp_1,BrokerId_1,ServiceTime_1,Cpu_1,QueueLength_1,Memory_1,AverageServiceTime_1,InterArrivalTime_1,QueueTime_1,AverageQueueTime_1,Throughput_1,AverageThroughput_2,ResponseTime_2,Timestamp_2,BrokerId_2,ServiceTime_2,Cpu_2,QueueLength_2,Memory_2,AverageServiceTime_2,InterArrivalTime_2,QueueTime_2,AverageQueueTime_2,Throughput_2
0,2024-11-30 17:35:54,localhost:50003,info,Advertisement(apple > 0.25195215418436445) 194f50a8-90e8-45b7-8c3c-bed7fb62e514,3,3.242796,0.308376,2024-11-30 17:35:54.0443138 +0900 KST m=+7.150063301,3,0.000000,71.543500,0,22896640,0.000000,0.000000,0.308376,0.308376,3.242796,,0.311918,2024-11-30 17:35:53.3877958 +0900 KST m=+6.508136701,2.0,0.000000,53.477604,0,23056384,,0.000000,0.311918,,3.205973,,0.303324,2024-11-30 17:35:52.7619299 +0900 KST m=+5.818396501,1.0,0.000000,64.282521,0,22675456,,0.000000,0.303324,,3.296801
1,2024-11-30 17:35:56,localhost:50003,info,Advertisement(apple > 0.8905283893938075) cd793b01-1cf5-4817-98b8-2f6804b7d137,2,1.077289,0.925696,2024-11-30 17:35:56.6000112 +0900 KST m=+9.705760701,3,0.622440,71.700718,0,23171072,0.622440,1.280409,0.303256,0.305816,1.080268,,0.594516,2024-11-30 17:35:55.5773986 +0900 KST m=+8.676233501,4.0,0.493589,66.722926,0,23662592,,0.559047,0.100927,,1.682040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-11-30 17:35:59,localhost:50003,info,Advertisement(apple > 0.9865251742875035) 4c923870-0d6b-4411-b9be-0ccdd3e6031d,3,1.013538,1.052808,2024-11-30 17:35:59.0810738 +0900 KST m=+12.186823301,3,0.723655,70.441196,0,23486464,0.673048,1.671994,0.329153,0.313595,0.949840,,0.228452,2024-11-30 17:35:58.3561582 +0900 KST m=+11.454993101,4.0,0.115075,69.896775,0,24137728,,0.971101,0.113377,,4.377291,,0.267956,2024-11-30 17:35:57.813104 +0900 KST m=+10.936167001,5.0,0.141435,67.067395,0,22700032,,0.944930,0.126521,,3.731953
3,2024-11-30 17:36:01,localhost:50003,info,Advertisement(apple > 0.6275339485574908) c5c491be-e864-4cc9-a729-803843b160ac,2,0.962491,1.146745,2024-11-30 17:36:01.4125202 +0900 KST m=+14.518269701,3,0.820683,64.739492,0,23744512,0.722260,1.837630,0.326061,0.316711,0.872034,,0.659852,2024-11-30 17:36:00.5385549 +0900 KST m=+13.637389801,4.0,0.518791,67.996619,0,24391680,,1.207823,0.141062,,1.515490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-11-30 17:36:06,localhost:50003,info,Advertisement(apple > 0.9646406184803333) 09eae1c4-2297-4467-914f-5d78ad86c9ee,2,0.937117,1.154278,2024-11-30 17:36:06.3848418 +0900 KST m=+19.490591301,3,0.823665,58.656242,0,23986176,0.747611,2.463658,0.330613,0.319492,0.866342,,0.260359,2024-11-30 17:36:05.664143 +0900 KST m=+18.762977901,4.0,0.137538,61.591855,0,24760320,,1.597563,0.122821,,3.840855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6966,2024-11-30 19:35:42,localhost:50003,info,Advertisement(apple > 0.3009694439421689) 68e848c3-aa65-46a7-8469-031f0cb1904b,3,0.021763,0.715067,2024-11-30 19:35:42.810599 +0900 KST m=+7195.916348501,3,0.514997,52.598151,0,42082304,0.814160,1.031847,0.200070,45.135267,1.398471,,3528.266437,2024-11-30 19:35:42.2959388 +0900 KST m=+7195.394773701,4.0,0.100027,64.071701,33080,77500416,,0.105465,3528.166410,,0.000283,,0.220075,2024-11-30 18:36:53.8174627 +0900 KST m=+3666.940525701,5.0,0.109571,64.559301,1,39182336,,1.050810,0.110505,,4.543899
6967,2024-11-30 19:35:43,localhost:50003,info,Advertisement(apple > 0.9005249814991677) ca13b4e1-5f30-4f7b-bfa1-233d22c312c5,1,0.021766,1.024097,2024-11-30 19:35:43.3247387 +0900 KST m=+7196.430488201,3,0.510554,52.596347,0,42082304,0.814116,1.031727,0.513543,45.128863,0.976470,NaN,Na

### Timestamp_n 간 차이 값 구하기

In [9]:
hop_df = preprocess_hop_log.process_timestamp(hop_df)

c:\Users\SJSJ\Desktop\wowsan_ai\preprocess_informer\preprocess_hop_log.py:89: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  m_values_df = hop_df[timestamp_cols].applymap(extract_m_value)
c:\Users\SJSJ\Desktop\wowsan_ai\preprocess_informer\preprocess_hop_log.py:94: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.notna(row[i+1]):
c:\Users\SJSJ\Desktop\wowsan_ai\preprocess_informer\preprocess_hop_log.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  next_value = row[i+1]
c:\Users\SJSJ\Desktop\wowsan_ai\preprocess_informer\preprocess_hop_log.py:96: FutureWarning: Series.__geti

In [10]:
hop_df

,time,Node,level,msg,HopCount,AverageThroughput_0,ResponseTime_0,BrokerId_0,ServiceTime_0,Cpu_0,QueueLength_0,Memory_0,AverageServiceTime_0,InterArrivalTime_0,QueueTime_0,AverageQueueTime_0,Throughput_0,AverageThroughput_1,ResponseTime_1,BrokerId_1,ServiceTime_1,Cpu_1,QueueLength_1,Memory_1,AverageServiceTime_1,InterArrivalTime_1,QueueTime_1,AverageQueueTime_1,Throughput_1,AverageThroughput_2,ResponseTime_2,BrokerId_2,ServiceTime_2,Cpu_2,QueueLength_2,Memory_2,AverageServiceTime_2,InterArrivalTime_2,QueueTime_2,AverageQueueTime_2,Throughput_2,Timestamp_0,Timestamp_1,Timestamp_2
0,2024-11-30 17:35:54,localhost:50003,info,Advertisement(apple > 0.25195215418436445) 194f50a8-90e8-45b7-8c3c-bed7fb62e514,3,3.242796,0.308376,3,0.000000,71.543500,0,22896640,0.000000,0.000000,0.308376,0.308376,3.242796,,0.311918,2.0,0.000000,53.477604,0,23056384,,0.000000,0.311918,,3.205973,,0.303324,1.0,0.000000,64.282521,0,22675456,,0.000000,0.303324,,3.296801,0.641927,0.689740,0.0
1,2024-11-30 17:35:56,localhost:50003,info,Advertisement(apple > 0.8905283893938075) cd793b01-1cf5-4817-98b8-2f6804b7d137,2,1.077289,0.925696,3,0.622440,71.700718,0,23171072,0.622440,1.280409,0.303256,0.305816,1.080268,,0.594516,4.0,0.493589,66.722926,0,23662592,,0.559047,0.100927,,1.682040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.029527,0.000000,0.0
2,2024-11-30 17:35:59,localhost:50003,info,Advertisement(apple > 0.9865251742875035) 4c923870-0d6b-4411-b9be-0ccdd3e6031d,3,1.013538,1.052808,3,0.723655,70.441196,0,23486464,0.673048,1.671994,0.329153,0.313595,0.949840,,0.228452,4.0,0.115075,69.896775,0,24137728,,0.971101,0.113377,,4.377291,,0.267956,5.0,0.141435,67.067395,0,22700032,,0.944930,0.126521,,3.731953,0.731830,0.518826,0.0
3,2024-11-30 17:36:01,localhost:50003,info,Advertisement(apple > 0.6275339485574908) c5c491be-e864-4cc9-a729-803843b160ac,2,0.962491,1.146745,3,0.820683,64.739492,0,23744512,0.722260,1.837630,0.326061,0.316711,0.872034,,0.659852,4.0,0.518791,67.996619,0,24391680,,1.207823,0.141062,,1.515490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.880880,0.000000,0.0
4,2024-11-30 17:36:06,localhost:50003,info,Advertisement(apple > 0.9646406184803333) 09eae1c4-2297-4467-914f-5d78ad86c9ee,2,0.937117,1.154278,3,0.823665,58.656242,0,23986176,0.747611,2.463658,0.330613,0.319492,0.866342,,0.260359,4.0,0.137538,61.591855,0,24760320,,1.597563,0.122821,,3.840855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.727613,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6966,2024-11-30 19:35:42,localhost:50003,info,Advertisement(apple > 0.3009694439421689) 68e848c3-aa65-46a7-8469-031f0cb1904b,3,0.021763,0.715067,3,0.514997,52.598151,0,42082304,0.814160,1.031847,0.200070,45.135267,1.398471,,3528.266437,4.0,0.100027,64.071701,33080,77500416,,0.105465,3528.166410,,0.000283,,0.220075,5.0,0.109571,64.559301,1,39182336,,1.050810,0.110505,,4.543899,0.521575,3528.454248,0.0
6967,2024-11-30 19:35:43,localhost:50003,info,Advertisement(apple > 0.9005249814991677) ca13b4e1-5f30-4f7b-bfa1-233d22c312c5,1,0.021766,1.024097,3,0.510554,52.596347,0,42082304,0.814116,1.031727,0.513543,45.128863,0.976470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.0
6968,2024-11-30 19:35:44,localhost:50003,info,Advertisement(apple > 0.019574597050384904) d05af190-75cb-492f-991e-79524e469054,3,0.021769,1.531467,3,0.830105,52.596375,0,42082304,0.814118,1.031671,0.701362,45.122488,0.652969,,4.922580,2.0,0.624734,52.405883,6,42475520,,1.103614,4.297847,,0.203145,,0.618323,1.0,0.309602,52.646949,1,41234432,,0.965343,0.308721,,1.617278,1.001588,4.670128,0.0
6969,2024-11-30 19:35:45,localhost:50003,info,Advertisement(apple > 0.011633017756757645) 2d58c030-75b9-44a8-ae89-17f2651128f3,3,0.021772,0.718266,3,0.512855,52.595444,0,42082304,0.814075,1.031791,0.205411,45.116044,1.392242,,5.538047,2.0,0.

In [11]:
# 폴더 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

hop_df.to_csv(path_mergeddf+'hop_df.csv', date_format='%Y-%m-%d %H:%M:%S', index=False)


## Tick Log 전처리

In [12]:
# json 파일을 열고 각 줄을 개별적으로 파싱
# path = '../raw_data/tickLogger/116/'
data = []
with open(path_ticklog + '50002_tick.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터를 DataFrame으로 변환
tick_df_raw = pd.DataFrame(data)
type(tick_df_raw)

pandas.core.frame.DataFrame

In [13]:
tick_df_raw

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,Cpu,InterArrivalTime,Memory,MessageCount,QueueLength,QueueTime,ResponseTime,ServiceTime,Throughput,TotalArrival Time,level,msg,time
0,0.000000,0.000000,0.000000,False,36.187652,0.000000,22102016,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:47
1,0.000000,0.000000,0.000000,False,49.796978,0.000000,22859776,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:48
2,0.000000,0.000000,0.000000,False,48.409659,0.000000,22872064,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:49
3,0.000000,0.000000,0.000000,False,49.798846,0.000000,22888448,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:50
4,0.000000,0.000000,0.000000,False,51.082387,0.000000,22896640,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,1.719835,0.624574,0.426547,False,52.406277,1.103558,42475520,6514,7,4.727124,5.352066,0.624943,0.186844,7188.577972,info,Performance Metrics,2024-11-30 19:35:41
7195,1.720279,0.624575,0.426466,False,52.406588,1.103558,42475520,6514,5,4.605386,5.230120,0.624734,0.191200,7188.577972,info,Performance Metrics,2024-11-30 19:35:42
7196,1.721150,0.624575,0.426307,False,52.405824,1.103524,42475520,6516,6,4.814553,5.439163,0.624609,0.183852,7190.562693,info,Performance Metrics,2024-11-30 19:35:43
7197,1.721622,0.624574,0.426222,False,52.404011,1.103524,42475520,6516,4,4.791871,5.412606,0.620735,0.184754,7190.562693,info,Performance Metrics,2024-11-30 19:35:44


In [14]:
# merged_df = tick_df.set_index('time').combine_first(hop_df.set_index('time')).reset_index()

In [15]:
merged_df = merge_hop_tick.merge_dataframes(hop_df, tick_df_raw)

In [16]:
merged_df

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,Cpu_0,InterArrivalTime_0,Memory_0,MessageCount,QueueLength_0,QueueTime_0,ResponseTime_0,ServiceTime_0,Throughput_0,TotalArrival Time,level,msg,time,Node,HopCount,AverageThroughput_0,BrokerId_0,AverageServiceTime_0,AverageQueueTime_0,AverageThroughput_1,ResponseTime_1,BrokerId_1,ServiceTime_1,Cpu_1,QueueLength_1,Memory_1,AverageServiceTime_1,InterArrivalTime_1,QueueTime_1,AverageQueueTime_1,Throughput_1,AverageThroughput_2,ResponseTime_2,BrokerId_2,ServiceTime_2,Cpu_2,QueueLength_2,Memory_2,AverageServiceTime_2,InterArrivalTime_2,QueueTime_2,AverageQueueTime_2,Throughput_2,Timestamp_0,Timestamp_1,Timestamp_2
0,0.000000,0.000000,0.000000,False,36.187652,0.000000,22102016,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,False,49.796978,0.000000,22859776,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,0.000000,0.000000,False,48.409659,0.000000,22872064,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,0.000000,0.000000,False,49.798846,0.000000,22888448,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000000,0.000000,0.000000,False,51.082387,0.000000,22896640,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-30 17:35:51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9500,1.719835,0.624574,0.426547,False,52.406277,1.103558,42475520,6514,7,4.727124,5.352066,0.624943,0.186844,7188.577972,info,Performance Metrics,2024-11-30 19:35:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9501,1.720279,0.624575,0.426466,False,52.598151,1.031847,42082304,6514,0,0.200070,0.715067,0.514997,1.398471,7188.577972,info,Advertisement(apple > 0.3009694439421689) 68e848c3-aa65-46a7-8469-031f0cb1904b,2024-11-30 19:35:42,localhost:50003,3.0,0.021763,3.0,0.814160,45.135267,,3528.266437,4.0,0.100027,64.071701,33080,77500416,,0.105465,3528.166410,,0.000283,,0.220075,5.0,0.109571,64.559301,1,39182336,,1.050810,0.110505,,4.543899,0.521575,3528.454248,0.0
9502,1.721150,0.624575,0.426307,False,52.596347,1.031727,42082304,6516,0,0.513543,1.024097,0.510554,0.976470,7190.562693,info,Advertisement(apple > 0.9005249814991677) ca13b4e1-5f30-4f7b-bfa1-233d22c312c5,2024-11-30 19:35:43,localhost:50003,1.0,0.021766,3.0,0.814116,45.128863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.0
9503,1.721622,0.624574,0.426222,False,52.596375,1.031671,42082304,6516,0,0.701362,1.531467,0.830105,0.652969,7190.562693,info,Advertisement(apple > 0.019574597050384904) d05af190-75cb-492f-991e-79524e469054,2024-11-30 19:35:44,localhost:50003,3.0,0.021769,3.0,0.814118,45.122488,,4.922580,2.0,0.624734,52.405883,6,42475520,,1.103614,4.297847,,0.203145,,0.618323,1.0,0.309602,52.646949,1,41234432,,0.965343,0.308721,,1.617278,1.001588,4.670128,0.0


In [17]:
# Bottleneck 컬럼의 False는 0으로, True는 1로 변환
merged_df['Bottleneck'] = merged_df['Bottleneck'].astype('float64')


In [18]:
# merged_df 컬럼 확인
merged_df.columns
# len(merged_df.columns)

Index(['AverageQueueTime', 'AverageServiceTime', 'AverageThroughput',
       'Bottleneck', 'Cpu_0', 'InterArrivalTime_0', 'Memory_0', 'MessageCount',
       'QueueLength_0', 'QueueTime_0', 'ResponseTime_0', 'ServiceTime_0',
       'Throughput_0', 'TotalArrival Time', 'level', 'msg', 'time', 'Node',
       'HopCount', 'AverageThroughput_0', 'BrokerId_0', 'AverageServiceTime_0',
       'AverageQueueTime_0', 'AverageThroughput_1', 'ResponseTime_1',
       'BrokerId_1', 'ServiceTime_1', 'Cpu_1', 'QueueLength_1', 'Memory_1',
       'AverageServiceTime_1', 'InterArrivalTime_1', 'QueueTime_1',
       'AverageQueueTime_1', 'Throughput_1', 'AverageThroughput_2',
       'ResponseTime_2', 'BrokerId_2', 'ServiceTime_2', 'Cpu_2',
       'QueueLength_2', 'Memory_2', 'AverageServiceTime_2',
       'InterArrivalTime_2', 'QueueTime_2', 'AverageQueueTime_2',
       'Throughput_2', 'Timestamp_0', 'Timestamp_1', 'Timestamp_2'],
      dtype='object')

### 학습에 필요한 컬럼만 남긴다

In [19]:
# 학습에 불필요한 컬럼 제거
# 불필요한 컬럼 리스트
drop_columns = [
    'level',
    'msg',
    'Node',
    # 'TotalArrivalTime',
    'MessageCount',
    # 'Timestamp'
]

# 불필요한 컬럼 제거
merged_df.drop(columns=drop_columns, inplace=True)

In [20]:
merged_df = merged_df.rename(columns={'time': 'date'})

In [21]:
merged_df  

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,Cpu_0,InterArrivalTime_0,Memory_0,QueueLength_0,QueueTime_0,ResponseTime_0,ServiceTime_0,Throughput_0,TotalArrival Time,date,HopCount,AverageThroughput_0,BrokerId_0,AverageServiceTime_0,AverageQueueTime_0,AverageThroughput_1,ResponseTime_1,BrokerId_1,ServiceTime_1,Cpu_1,QueueLength_1,Memory_1,AverageServiceTime_1,InterArrivalTime_1,QueueTime_1,AverageQueueTime_1,Throughput_1,AverageThroughput_2,ResponseTime_2,BrokerId_2,ServiceTime_2,Cpu_2,QueueLength_2,Memory_2,AverageServiceTime_2,InterArrivalTime_2,QueueTime_2,AverageQueueTime_2,Throughput_2,Timestamp_0,Timestamp_1,Timestamp_2
0,0.000000,0.000000,0.000000,0.0,36.187652,0.000000,22102016,0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-30 17:35:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,0.0,49.796978,0.000000,22859776,0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-30 17:35:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,0.000000,0.000000,0.0,48.409659,0.000000,22872064,0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-30 17:35:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,0.000000,0.000000,0.0,49.798846,0.000000,22888448,0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-30 17:35:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000000,0.000000,0.000000,0.0,51.082387,0.000000,22896640,0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-30 17:35:51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9500,1.719835,0.624574,0.426547,0.0,52.406277,1.103558,42475520,7,4.727124,5.352066,0.624943,0.186844,7188.577972,2024-11-30 19:35:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9501,1.720279,0.624575,0.426466,0.0,52.598151,1.031847,42082304,0,0.200070,0.715067,0.514997,1.398471,7188.577972,2024-11-30 19:35:42,3.0,0.021763,3.0,0.814160,45.135267,,3528.266437,4.0,0.100027,64.071701,33080,77500416,,0.105465,3528.166410,,0.000283,,0.220075,5.0,0.109571,64.559301,1,39182336,,1.050810,0.110505,,4.543899,0.521575,3528.454248,0.0
9502,1.721150,0.624575,0.426307,0.0,52.596347,1.031727,42082304,0,0.513543,1.024097,0.510554,0.976470,7190.562693,2024-11-30 19:35:43,1.0,0.021766,3.0,0.814116,45.128863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.0
9503,1.721622,0.624574,0.426222,0.0,52.596375,1.031671,42082304,0,0.701362,1.531467,0.830105,0.652969,7190.562693,2024-11-30 19:35:44,3.0,0.021769,3.0,0.814118,45.122488,,4.922580,2.0,0.624734,52.405883,6,42475520,,1.103614,4.297847,,0.203145,,0.618323,1.0,0.309602,52.646949,1,41234432,,0.965343,0.308721,,1.617278,1.001588,4.670128,0.0


In [22]:
# # Timestamp_0 컬럼 제거
# merged_df.drop(columns=['Timestamp_0'], inplace=True)

# # 1500 행 이후의 데이터를 제거
# merged_df = merged_df[:1500]

# 모든 값이 NaN인 컬럼 이름 출력 후 해당 컬럼 제거
print(merged_df.columns[merged_df.isna().all()].tolist())
merged_df.drop(columns=merged_df.columns[merged_df.isna().all()].tolist(), inplace=True)


[]


In [23]:
# merge_df 의 컬럼 출력
merged_df.columns

Index(['AverageQueueTime', 'AverageServiceTime', 'AverageThroughput',
       'Bottleneck', 'Cpu_0', 'InterArrivalTime_0', 'Memory_0',
       'QueueLength_0', 'QueueTime_0', 'ResponseTime_0', 'ServiceTime_0',
       'Throughput_0', 'TotalArrival Time', 'date', 'HopCount',
       'AverageThroughput_0', 'BrokerId_0', 'AverageServiceTime_0',
       'AverageQueueTime_0', 'AverageThroughput_1', 'ResponseTime_1',
       'BrokerId_1', 'ServiceTime_1', 'Cpu_1', 'QueueLength_1', 'Memory_1',
       'AverageServiceTime_1', 'InterArrivalTime_1', 'QueueTime_1',
       'AverageQueueTime_1', 'Throughput_1', 'AverageThroughput_2',
       'ResponseTime_2', 'BrokerId_2', 'ServiceTime_2', 'Cpu_2',
       'QueueLength_2', 'Memory_2', 'AverageServiceTime_2',
       'InterArrivalTime_2', 'QueueTime_2', 'AverageQueueTime_2',
       'Throughput_2', 'Timestamp_0', 'Timestamp_1', 'Timestamp_2'],
      dtype='object')

In [24]:
new_order = ['date', 'Bottleneck', 'HopCount', 
             'Timestamp_0', 'BrokerId_0', 'Cpu_0', 'Memory_0', 'InterArrivalTime_0', 'QueueLength_0', 'QueueTime_0', 'ServiceTime_0', 'ResponseTime_0', 'Throughput_0',
             'Timestamp_1',  'BrokerId_1', 'Cpu_1', 'Memory_1', 'InterArrivalTime_1', 'QueueLength_1',  'QueueTime_1', 'ServiceTime_1', 'ResponseTime_1', 'Throughput_1',
             'Timestamp_2', 'BrokerId_2', 'Cpu_2', 'Memory_2', 'InterArrivalTime_2', 'QueueLength_2', 'QueueTime_2', 'ServiceTime_2', 'ResponseTime_2', 'Throughput_2']

# 새로운 순서로 데이터프레임을 재정렬합니다
merged_df = merged_df[new_order]

In [25]:
drop_columns = [ 
    'Bottleneck',
#     'BrokerId_0',
#     'BrokerId_1',
#     'BrokerId_2'
]

# # 불필요한 컬럼 제거
merged_df.drop(columns=drop_columns, inplace=True)

C:\Users\SJSJ\AppData\Local\Temp\ipykernel_38152\2104203700.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.drop(columns=drop_columns, inplace=True)


In [26]:
# return all unique values of merged_df columns
for col in merged_df.columns:
    print(col)
    print(merged_df[col].unique())
    print()


date
['2024-11-30 17:35:47' '2024-11-30 17:35:48' '2024-11-30 17:35:49' ...
 '2024-11-30 19:35:43' '2024-11-30 19:35:44' '2024-11-30 19:35:45']

HopCount
[nan  3.  2.  1.]

Timestamp_0
[      nan 0.6419266 1.0295272 ... 0.5215748 1.0015882 0.5059008]

BrokerId_0
[nan  3.]

Cpu_0
[36.18765162915534 49.796978022402534 48.40965853526813 ... '52.596347'
 '52.596375' '52.595444']

Memory_0
[22102016 22859776 22872064 ... '42078208' '42082304' 42475520]

InterArrivalTime_0
['0.000000' '1.280409' '1.969363' ... '1.031847' '1.031671' '1.031791']

QueueLength_0
[0 '0' '1' '2' '3' '9' '15' '17' '23' '32' '34' '41' '48' '50' '57' '65'
 '70' '69' '68' '67' '66' '64' '63' '62' '61' '60' '59' '58' '56' '55'
 '54' '53' '52' '51' '49' '47' '46' '45' '44' '43' '42' '40' '39' '38'
 '37' '36' '35' '33' '31' '30' '29' '28' '27' '26' '25' '24' '22' '21'
 '20' '19' '18' '16' '14' '13' '12' '11' '10' '8' '7' '6' '5' '4' 25 22 13
 10 8 6 1 '73' '78' '80' '85' '89' '94' '100' '102' '108' '107' '106'
 '105' '10

In [27]:
# NaN 값을 0으로 대체
merged_df.fillna(0, inplace=True)

C:\Users\SJSJ\AppData\Local\Temp\ipykernel_38152\1254312440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.fillna(0, inplace=True)


In [28]:
# fill_nan.fill_nan(merged_df)

In [29]:
merged_df

,date,HopCount,Timestamp_0,BrokerId_0,Cpu_0,Memory_0,InterArrivalTime_0,QueueLength_0,QueueTime_0,ServiceTime_0,ResponseTime_0,Throughput_0,Timestamp_1,BrokerId_1,Cpu_1,Memory_1,InterArrivalTime_1,QueueLength_1,QueueTime_1,ServiceTime_1,ResponseTime_1,Throughput_1,Timestamp_2,BrokerId_2,Cpu_2,Memory_2,InterArrivalTime_2,QueueLength_2,QueueTime_2,ServiceTime_2,ResponseTime_2,Throughput_2
0,2024-11-30 17:35:47,0.0,0.000000,0.0,36.187652,22102016,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
1,2024-11-30 17:35:48,0.0,0.000000,0.0,49.796978,22859776,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
2,2024-11-30 17:35:49,0.0,0.000000,0.0,48.409659,22872064,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
3,2024-11-30 17:35:50,0.0,0.000000,0.0,49.798846,22888448,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
4,2024-11-30 17:35:51,0.0,0.000000,0.0,51.082387,22896640,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9500,2024-11-30 19:35:41,0.0,0.000000,0.0,52.406277,42475520,1.103558,7,4.727124,0.624943,5.352066,0.186844,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
9501,2024-11-30 19:35:42,3.0,0.521575,3.0,52.598151,42082304,1.031847,0,0.200070,0.514997,0.715067,1.398471,3528.454248,4.0,64.071701,77500416,0.105465,33080,3528.166410,0.100027,3528.266437,0.000283,0.0,5.0,64.559301,39182336,1.050810,1,0.110505,0.109571,0.220075,4.543899
9502,2024-11-30 19:35:43,1.0,0.000000,3.0,52.596347,42082304,1.031727,0,0.513543,0.510554,1.024097,0.976470,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
9503,2024-11-30 19:35:44,3.0,1.001588,3.0,52.596375,42082304,1.031671,0,0.701362,0.830105,1.531467,0.652969,4.670128,2.0,52.405883,42475520,1.103614,6,4.297847,0.624734,4.922580,0.203145,0.0,1.0,52.646949,41234432,0.965343,1,0.308721,0.309602,0.618323,1.617278


In [30]:
# 각 컬럼별로 unique 값이 하나인 컬럼 삭제
for col in merged_df.columns:
    if len(merged_df[col].unique()) == 1:
        print(col)
        merged_df = merged_df.drop(col, axis=1)

Timestamp_2


In [31]:
# save to csv
# 디렉토리가 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

merged_df.to_csv(path_mergeddf+'merged_df.csv', date_format='%Y-%m-%d %H:%M:%S', index=False)


In [32]:
merged_df

,date,HopCount,Timestamp_0,BrokerId_0,Cpu_0,Memory_0,InterArrivalTime_0,QueueLength_0,QueueTime_0,ServiceTime_0,ResponseTime_0,Throughput_0,Timestamp_1,BrokerId_1,Cpu_1,Memory_1,InterArrivalTime_1,QueueLength_1,QueueTime_1,ServiceTime_1,ResponseTime_1,Throughput_1,BrokerId_2,Cpu_2,Memory_2,InterArrivalTime_2,QueueLength_2,QueueTime_2,ServiceTime_2,ResponseTime_2,Throughput_2
0,2024-11-30 17:35:47,0.0,0.000000,0.0,36.187652,22102016,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
1,2024-11-30 17:35:48,0.0,0.000000,0.0,49.796978,22859776,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
2,2024-11-30 17:35:49,0.0,0.000000,0.0,48.409659,22872064,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
3,2024-11-30 17:35:50,0.0,0.000000,0.0,49.798846,22888448,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
4,2024-11-30 17:35:51,0.0,0.000000,0.0,51.082387,22896640,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9500,2024-11-30 19:35:41,0.0,0.000000,0.0,52.406277,42475520,1.103558,7,4.727124,0.624943,5.352066,0.186844,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
9501,2024-11-30 19:35:42,3.0,0.521575,3.0,52.598151,42082304,1.031847,0,0.200070,0.514997,0.715067,1.398471,3528.454248,4.0,64.071701,77500416,0.105465,33080,3528.166410,0.100027,3528.266437,0.000283,5.0,64.559301,39182336,1.050810,1,0.110505,0.109571,0.220075,4.543899
9502,2024-11-30 19:35:43,1.0,0.000000,3.0,52.596347,42082304,1.031727,0,0.513543,0.510554,1.024097,0.976470,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
9503,2024-11-30 19:35:44,3.0,1.001588,3.0,52.596375,42082304,1.031671,0,0.701362,0.830105,1.531467,0.652969,4.670128,2.0,52.405883,42475520,1.103614,6,4.297847,0.624734,4.922580,0.203145,1.0,52.646949,41234432,0.965343,1,0.308721,0.309602,0.618323,1.617278


In [33]:
# 컬럼별 datatype 확인
merged_df.dtypes

date                   object
HopCount              float64
Timestamp_0           float64
BrokerId_0            float64
Cpu_0                  object
Memory_0               object
InterArrivalTime_0     object
QueueLength_0          object
QueueTime_0            object
ServiceTime_0          object
ResponseTime_0         object
Throughput_0           object
Timestamp_1           float64
BrokerId_1            float64
Cpu_1                  object
Memory_1               object
InterArrivalTime_1     object
QueueLength_1          object
QueueTime_1            object
ServiceTime_1          object
ResponseTime_1         object
Throughput_1           object
BrokerId_2            float64
Cpu_2                  object
Memory_2               object
InterArrivalTime_2     object
QueueLength_2          object
QueueTime_2            object
ServiceTime_2          object
ResponseTime_2         object
Throughput_2           object
dtype: object